# install

In [1]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!pip install ctgan==0.6.0
!pip install filterpy
!pip install Autogluon

In [3]:
import ctgan
ctgan.__version__

'0.6.0'

In [5]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import warnings
import joblib
import os
import torch
import random
import numpy as np
import glob
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
from tqdm import tqdm
from scipy.signal import butter, lfilter

warnings.filterwarnings('ignore')

# def

In [6]:
def LPF(series, low, order=1):
   
    b, a = butter(
                  N = order,
                  Wn = low,
                  btype = 'low',
                  )
    lpf_series = lfilter(b, a, series)
    
    return lpf_series

def preprocessing_2(data):
    data['obs_time'] = data.index % 24  # 시간통일
    col_list = data.columns
    for i in range(0, len(col_list)):
        col = col_list[i]
        if '누적' in col:
            data[col] = data.groupby((data.obs_time == 0).cumsum()).agg(
                col_list[i-1]).cumsum()
    return data

def filtering(df):

    f1 = df[df['co2관측치']>578]['Case'].unique().tolist()
    f2 = df[((df['내부습도관측치']<74.4)&(df['내부습도관측치']>73.6))|((df['내부습도관측치']<30)&(df['내부습도관측치']>27))|((df['내부습도관측치']<54.5)&(df['내부습도관측치']>54.4)) ]['Case'].unique().tolist()
    f3 = df[(df['일간누적분무량']<150)]['Case'].unique().tolist()
    
    d1 = df[(df['시간당분무량']>2100)]['Case'].unique().tolist()
    d2 = df[(df['내부온도관측치']>31.124)&(df['내부온도관측치']<31.126)]['Case'].unique().tolist()
    d3 = df[(df['co2관측치']<510.1)&(df['co2관측치']>=510)]['Case'].unique().tolist()
    d4 = df[(df['시간당백색광량']>3094.0)&(df['시간당백색광량']<3094.2)]['Case'].unique().tolist()
    d5 = df[(df['시간당백색광량']>13400)&(df['시간당백색광량']<13400.5472)]['Case'].unique().tolist()

    return_arr = list(set(f1)&set(f2)&set(f3))
    for i in set(d1+d2+d3+d4+d5):
        if i in return_arr :
            return_arr.remove(i)

    return return_arr

def makedatasets(input_path):

    test = preprocessing_2(input_path) #input_path : 생성될 데이터셋 path
    print(test)
    test['05시내부온도관측치누적'] = 0
    test['19시내부온도관측치누적'] = 0
    test['23시내부온도관측치누적'] = 0

    test['05시내부습도관측치누적'] = 0
    test['19시내부습도관측치누적'] = 0
    test['23시내부습도관측치누적'] = 0

    test['05시co2관측치누적'] = 0
    test['19시co2관측치누적'] = 0
    test['23시co2관측치누적'] = 0

    test['05시ec관측치누적'] = 0
    test['19시ec관측치누적'] = 0
    test['23시ec관측치누적'] = 0

    test['05시분무량누적'] = 0
    test['19시분무량누적'] = 0
    test['23시분무량누적'] = 0

    test['05시백색광누적'] = 0
    test['19시백색광누적'] = 0
    test['23시백색광누적'] = 0

    test['05시적색광누적'] = 0
    test['19시적색광누적'] = 0
    test['23시적색광누적'] = 0

    test['05시청색광누적'] = 0
    test['19시청색광누적'] = 0
    test['23시청색광누적'] = 0

    test['05시총광량누적'] = 0
    test['19시총광량누적'] = 0
    test['23시총광량누적'] = 0
    
    testfilter = filtering(test)

    test = test.drop(test.filter(regex='일간누적').columns, axis=1)

    test_new = test

    test['시간대'] = 0
    test['시간대'][(test['obs_time'] >= 0) & (test['obs_time'] <= 5)] = 1
    test['시간대'][(test['obs_time'] > 5) & (test['obs_time'] < 20)] = 2
    test['시간대'][(test['obs_time'] >= 20) & (test['obs_time'] <= 23)] = 3

    #########################################################교체#####################

    test['hypothesis1'] = 0
    print(test.shape)
    for i in range(22,len(test),24):
        s = test.iloc[i,6] + test.iloc[i+1,6] * ((test.iloc[i, 3] + test.iloc[i+1,3]) / 2)
        test.iloc[i, 40] = s



    test['hypothesis2'] = 0
    for i in range(22,len(test),24):
        s = test.iloc[i+1,6] * (test.iloc[i+1,3])
        test.iloc[i, 41] = s



    test['hypothesis3'] = 0
    for i in range(22,len(test),24):
        s = (test.iloc[i,6] + test.iloc[i+1,6]) * ((test.iloc[i, 3] + test.iloc[i+1,3]) / 2)
        test.iloc[i, 42] = s

    ###################################################################################################################

    test_x = test
    test_x = test_x.groupby(['DAT','Case','시간대']).sum().reset_index()


    test_x = test_x.sort_values(by=['Case','DAT','시간대'], axis=0).reset_index()
    test_x.drop(['index'], axis = 1, inplace=True)


    t = test_x.groupby(['Case'])['내부온도관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t.drop(['level_1'], axis = 1, inplace = True)
    test_x['내부온도관측치'] = t

    t2 = test_x.groupby(['Case'])['내부습도관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t2.drop(['level_1'], axis = 1, inplace = True)
    test_x['내부습도관측치'] = t2

    t3 = test_x.groupby(['Case'])['co2관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t3.drop(['level_1'], axis = 1, inplace = True)
    test_x['co2관측치'] = t3

    t4 = test_x.groupby(['Case'])['ec관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t4.drop(['level_1'], axis = 1, inplace = True)
    test_x['ec관측치'] = t4

    t5 = test_x.groupby(['Case'])['시간당분무량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t5.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당분무량'] = t5

    t6 = test_x.groupby(['Case'])['시간당백색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t6.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당백색광량'] = t6

    t7 = test_x.groupby(['Case'])['시간당적색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t7.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당적색광량'] = t7

    t8 = test_x.groupby(['Case'])['시간당청색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t8.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당청색광량'] = t8

    t9 = test_x.groupby(['Case'])['시간당총광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t9.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당총광량'] = t9

    test_new_x = test_new
    test_new_x = test_new_x.groupby(['DAT','Case']).sum().reset_index()

    test_new_x = test_new_x.sort_values(by=['Case','DAT'], axis=0).reset_index()
    test_new_x.drop(['index'], axis = 1, inplace=True)


    case_list = test_x['Case'].unique()

    for c in case_list : 
        if c in testfilter :
            continue
        test_x.loc[(test_x['Case'] == c ), 'hypothesis1'] = 0
        test_x.loc[(test_x['Case'] == c ), 'hypothesis2'] = 0
        test_x.loc[(test_x['Case'] == c ), 'hypothesis3'] = 0

    for i in range(len(test_new_x)):
        #온도
        test_new_x.iloc[i,12] = test_x.iloc[0 + (3*i),4]
        test_new_x.iloc[i,13] = test_x.iloc[1 + (3*i),4]
        test_new_x.iloc[i,14] = test_x.iloc[2 + (3*i),4]
        #습도
        test_new_x.iloc[i,15] = test_x.iloc[0 + (3*i),5]
        test_new_x.iloc[i,16] = test_x.iloc[1 + (3*i),5]
        test_new_x.iloc[i,17] = test_x.iloc[2 + (3*i),5]
        #co2
        test_new_x.iloc[i,18] = test_x.iloc[0 + (3*i),6]
        test_new_x.iloc[i,19] = test_x.iloc[1 + (3*i),6]
        test_new_x.iloc[i,20] = test_x.iloc[2 + (3*i),6]
        #ec
        test_new_x.iloc[i,21] = test_x.iloc[0 + (3*i),7]
        test_new_x.iloc[i,22] = test_x.iloc[1 + (3*i),7]
        test_new_x.iloc[i,23] = test_x.iloc[2 + (3*i),7]
        #분무
        test_new_x.iloc[i,24] = test_x.iloc[0 + (3*i),8]
        test_new_x.iloc[i,25] = test_x.iloc[1 + (3*i),8]
        test_new_x.iloc[i,26] = test_x.iloc[2 + (3*i),8]
        #백색
        test_new_x.iloc[i,27] = test_x.iloc[0 + (3*i),9]
        test_new_x.iloc[i,28] = test_x.iloc[1 + (3*i),9]
        test_new_x.iloc[i,29] = test_x.iloc[2 + (3*i),9]
        #적색
        test_new_x.iloc[i,30] = test_x.iloc[0 + (3*i),10]
        test_new_x.iloc[i,31] = test_x.iloc[1 + (3*i),10]
        test_new_x.iloc[i,32] = test_x.iloc[2 + (3*i),10]
        #청색
        test_new_x.iloc[i,33] = test_x.iloc[0 + (3*i),11]
        test_new_x.iloc[i,34] = test_x.iloc[1 + (3*i),11]
        test_new_x.iloc[i,35] = test_x.iloc[2 + (3*i),11]
        #총광
        test_new_x.iloc[i,36] = test_x.iloc[0 + (3*i),12]
        test_new_x.iloc[i,37] = test_x.iloc[1 + (3*i),12]
        test_new_x.iloc[i,38] = test_x.iloc[2 + (3*i),12]



    test_new_x.drop(['obs_time','시간대','내부온도관측치','내부습도관측치','co2관측치','ec관측치'], axis = 1, inplace = True)

    test_x = test_new_x

    test_x = test_x.drop(test_x.filter(regex='시간당').columns, axis=1)


    test_x['하루평균온도'] = (test_x['05시내부온도관측치누적'] + test_x['19시내부온도관측치누적'] + test_x['23시내부온도관측치누적']) / 3
    test_x['하루평균습도'] = (test_x['05시내부습도관측치누적'] + test_x['19시내부습도관측치누적'] + test_x['23시내부습도관측치누적']) / 3
    test_x['하루평균co2'] = (test_x['05시co2관측치누적'] + test_x['19시co2관측치누적'] + test_x['23시co2관측치누적']) / 3
    test_x['하루평균ec'] = (test_x['05시ec관측치누적'] + test_x['19시ec관측치누적'] + test_x['23시ec관측치누적']) / 3
    test_x['하루평균분무량'] = (test_x['05시분무량누적'] + test_x['19시분무량누적'] + test_x['23시분무량누적']) / 3
    test_x['하루평균백색광'] = (test_x['05시백색광누적'] + test_x['19시백색광누적'] + test_x['23시백색광누적']) / 3
    test_x['하루평균적색광'] = (test_x['05시적색광누적'] + test_x['19시적색광누적'] + test_x['23시적색광누적']) / 3
    test_x['하루평균청색광'] = (test_x['05시청색광누적'] + test_x['19시청색광누적'] + test_x['23시청색광누적']) / 3
    test_x['하루평균총광량'] = (test_x['05시총광량누적'] + test_x['19시총광량누적'] + test_x['23시총광량누적']) / 3

    test_x = test_x.drop(test_x.filter(regex = '총광').columns, axis =1)


    test_x['hypothesis4'] = (test_x['05시ec관측치누적']+1) * (test_x['05시분무량누적']+1)
    test_x['hypothesis5'] = (test_x['19시ec관측치누적']+1) * (test_x['19시분무량누적']+1)
    test_x['hypothesis6'] = (test_x['23시ec관측치누적']+1) * (test_x['23시분무량누적']+1)
    test_x['hypothesis7'] = (test_x['하루평균ec']+1) * (test_x['하루평균분무량']+1)

    test_x['적색_+_청색05'] = (test_x['05시적색광누적']) + (test_x['05시청색광누적'])
    test_x['적색_+_청색19'] = (test_x['19시적색광누적']) + (test_x['19시청색광누적'])
    test_x['적색_+_청색23'] = (test_x['23시적색광누적']) + (test_x['23시청색광누적'])
    test_x['적색_+_청색평균'] = (test_x['하루평균적색광']) + (test_x['하루평균청색광'])


    test_x['hypothesis123'] =  test_x['hypothesis1'] + test_x['hypothesis2'] + test_x['hypothesis3']
    test_x['hypothesis12'] =  test_x['hypothesis2'] + test_x['hypothesis1']
    test_x['hypothesis23'] =  test_x['hypothesis2'] + test_x['hypothesis3']
    test_x['hypothesis13'] =  test_x['hypothesis1'] + test_x['hypothesis3']
    test_x['hypothesis123'] =  test_x['hypothesis1'] + test_x['hypothesis2'] + test_x['hypothesis3']
    
    test_x.drop(test_x.filter(regex = '백색'), axis = 1, inplace=True)

    test_x.drop(test_x.filter(regex = '청색'), axis = 1, inplace=True)

    test_x.drop(['hypothesis1', 'hypothesis2', 'hypothesis3'],axis=1,inplace=True)


    basic_col = test_x.columns
    kal_test = test_x

    for i in range(2,34):
        test_x['kf_X_'+str(i)]=0

    for j in range(0,len(test_x.Case.unique())): 
        kal = test_x[test_x['Case'] == j]
        for i in (range(len(basic_col))):
            if((i == 0) | (i == 1)):
                continue
            current=0
            sum_c=[]
            z = kal.loc[:, kal.columns[i]]
            a = []           
            b = []         
            my_filter = KalmanFilter(dim_x=2,dim_z=1)
            my_filter.x = np.array([[2.],[0.]])     
            my_filter.F = np.array([[1.,1.], [0.,1.]])   
            my_filter.H = np.array([[1.,0.]])
            my_filter.P *= 1000.              
            my_filter.R = 5                     
            my_filter.Q = Q_discrete_white_noise(dim = 2,dt=.1,var=.5) 
            for k in z.values:
                my_filter.predict()
                my_filter.update(k)
                x = my_filter.x
                a.extend(x[0])
                b.append(k)
            sum_c=sum_c+a
            test_x['kf_X_'+str(i)][test_x['Case'] == j] = sum_c
    copy_test_x = test_x.copy()

    copy_test_x.drop(copy_test_x.filter(regex = 'kf').columns, axis = 1, inplace = True)
    print(copy_test_x)

    raw_cols_te = copy_test_x.columns
    new_test_x = pd.DataFrame()
    for i in range(0,len(copy_test_x.Case.unique())):
        target = copy_test_x[copy_test_x['Case'] == i]
        mean_arr = []
        median_arr = []
        pbar = (target.columns[:])
        for column in pbar:
            column_list = target[column].to_list()
            for i in range(7):
                if((i) == (len(column_list))):
                    break
                mean_arr.append(column_list[i])
                median_arr.append(column_list[i])
            for i in range(7, len(column_list)):
                mean_arr.append(float(np.mean(column_list[i-7:i])))
                median_arr.append(float(np.median(column_list[i-7:i])))
            target[f'{column}_mean_7'] = mean_arr
            target[f'{column}_median_7'] = median_arr
            mean_arr = []
            median_arr = []
        
        new_test_x = pd.concat([new_test_x, target], axis=0)
    test_x_moving_7 = new_test_x.drop(raw_cols_te, axis=1)

    test_x = pd.concat([test_x, test_x_moving_7], axis=1)

    raw_cols_te = copy_test_x.columns

    new_test_x = pd.DataFrame()

    for i in range(0,len(copy_test_x.Case.unique())):
        target = copy_test_x[copy_test_x['Case'] == i]
        mean_arr = []
        median_arr = []
        pbar = (target.columns[:])
        for column in pbar:
            column_list = target[column].to_list()
            for i in range(14):
                if((i) == (len(column_list))):
                    break
                mean_arr.append(column_list[i])
                median_arr.append(column_list[i])
            for i in range(14, len(column_list)):
                mean_arr.append(float(np.mean(column_list[i-14:i])))
                median_arr.append(float(np.median(column_list[i-14:i])))
            target[f'{column}_mean_14'] = mean_arr
            target[f'{column}_median_14'] = median_arr
            mean_arr = []
            median_arr = []
        
        new_test_x = pd.concat([new_test_x, target], axis=0)
    test_x_moving_14 = new_test_x.drop(raw_cols_te, axis=1)

    test_x = pd.concat([test_x, test_x_moving_14], axis=1)

    test_x_fil = test_x.iloc[:,1:34]

    new_test_x = pd.DataFrame()
    for i in range(0,len(copy_test_x.Case.unique())):
        target = test_x_fil[test_x_fil['Case'] == i]
        test_merge = LPF(target,0.1,1)
        test_merge = pd.DataFrame(test_merge)
        
        new_test_x = pd.concat([new_test_x, test_merge], axis=0)
    new_test_x = new_test_x.add_suffix('_LPF')
    new_test_x = new_test_x.reset_index(drop = True)
    test_x = pd.concat([test_x,new_test_x], axis = 1)

    test_x.drop(['0_LPF'],axis = 1, inplace=True)
    test_x.drop(test_x.filter(regex = 'Case'),axis=1, inplace=True)

    return test_x

In [7]:
def make_data(input_path, fix_path, day):
    test = preprocessing_2(input_path) 

    if day != 0 : 
        test = pd.concat([test,fix_path],axis=0)
    test = test.reset_index(drop = True)

    test['05시내부온도관측치누적'] = 0
    test['19시내부온도관측치누적'] = 0
    test['23시내부온도관측치누적'] = 0

    test['05시내부습도관측치누적'] = 0
    test['19시내부습도관측치누적'] = 0
    test['23시내부습도관측치누적'] = 0

    test['05시co2관측치누적'] = 0
    test['19시co2관측치누적'] = 0
    test['23시co2관측치누적'] = 0

    test['05시ec관측치누적'] = 0
    test['19시ec관측치누적'] = 0
    test['23시ec관측치누적'] = 0

    test['05시분무량누적'] = 0
    test['19시분무량누적'] = 0
    test['23시분무량누적'] = 0

    test['05시백색광누적'] = 0
    test['19시백색광누적'] = 0
    test['23시백색광누적'] = 0

    test['05시적색광누적'] = 0
    test['19시적색광누적'] = 0
    test['23시적색광누적'] = 0

    test['05시청색광누적'] = 0
    test['19시청색광누적'] = 0
    test['23시청색광누적'] = 0

    test['05시총광량누적'] = 0
    test['19시총광량누적'] = 0
    test['23시총광량누적'] = 0

    testfilter = filtering(test)

    test = test.drop(test.filter(regex='일간누적').columns, axis=1)

    test_new = test

    test['시간대'] = 0
    test['시간대'][(test['obs_time'] >= 0) & (test['obs_time'] <= 5)] = 1
    test['시간대'][(test['obs_time'] > 5) & (test['obs_time'] < 20)] = 2
    test['시간대'][(test['obs_time'] >= 20) & (test['obs_time'] <= 23)] = 3

    test['hypothesis1'] = 0
    for i in range(22,len(test),24):
        s = test.iloc[i,6] + test.iloc[i+1,6] * ((test.iloc[i, 3] + test.iloc[i+1,3]) / 2)
        test.iloc[i, 40] = s

    test['hypothesis2'] = 0
    for i in range(22,len(test),24):
        s = test.iloc[i+1,6] * (test.iloc[i+1,3])
        test.iloc[i, 41] = s


    test['hypothesis3'] = 0
    for i in range(22,len(test),24):
        s = (test.iloc[i,6] + test.iloc[i+1,6]) * ((test.iloc[i, 3] + test.iloc[i+1,3]) / 2)
        test.iloc[i, 42] = s

    ###################################################################################################################

    case_list = test['Case'].unique()

    for c in case_list : 
        if c in testfilter :
            continue
        test.loc[(test['Case'] == c ), 'hypothesis1'] = 0
        test.loc[(test['Case'] == c ), 'hypothesis2'] = 0
        test.loc[(test['Case'] == c ), 'hypothesis3'] = 0

    test_x = test
    test_x = test_x.groupby(['DAT','Case','시간대']).sum().reset_index()

    test_x = test_x.sort_values(by=['Case','DAT','시간대'], axis=0).reset_index()
    test_x.drop(['index'], axis = 1, inplace=True)


    t = test_x.groupby(['Case'])['내부온도관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t.drop(['level_1'], axis = 1, inplace = True)
    test_x['내부온도관측치'] = t

    t2 = test_x.groupby(['Case'])['내부습도관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t2.drop(['level_1'], axis = 1, inplace = True)
    test_x['내부습도관측치'] = t2

    t3 = test_x.groupby(['Case'])['co2관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t3.drop(['level_1'], axis = 1, inplace = True)
    test_x['co2관측치'] = t3

    t4 = test_x.groupby(['Case'])['ec관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t4.drop(['level_1'], axis = 1, inplace = True)
    test_x['ec관측치'] = t4

    t5 = test_x.groupby(['Case'])['시간당분무량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t5.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당분무량'] = t5

    t6 = test_x.groupby(['Case'])['시간당백색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t6.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당백색광량'] = t6

    t7 = test_x.groupby(['Case'])['시간당적색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t7.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당적색광량'] = t7

    t8 = test_x.groupby(['Case'])['시간당청색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t8.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당청색광량'] = t8

    t9 = test_x.groupby(['Case'])['시간당총광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
    t9.drop(['level_1'], axis = 1, inplace = True)
    test_x['시간당총광량'] = t9

    test_new_x = test_new
    test_new_x = test_new_x.groupby(['DAT','Case']).sum().reset_index()

    test_new_x = test_new_x.sort_values(by=['Case','DAT'], axis=0).reset_index()
    test_new_x.drop(['index'], axis = 1, inplace=True)


    for i in range(len(test_new_x)):
        #온도
        test_new_x.iloc[i,12] = test_x.iloc[0 + (3*i),4]
        test_new_x.iloc[i,13] = test_x.iloc[1 + (3*i),4]
        test_new_x.iloc[i,14] = test_x.iloc[2 + (3*i),4]
        #습도
        test_new_x.iloc[i,15] = test_x.iloc[0 + (3*i),5]
        test_new_x.iloc[i,16] = test_x.iloc[1 + (3*i),5]
        test_new_x.iloc[i,17] = test_x.iloc[2 + (3*i),5]
        #co2
        test_new_x.iloc[i,18] = test_x.iloc[0 + (3*i),6]
        test_new_x.iloc[i,19] = test_x.iloc[1 + (3*i),6]
        test_new_x.iloc[i,20] = test_x.iloc[2 + (3*i),6]
        #ec
        test_new_x.iloc[i,21] = test_x.iloc[0 + (3*i),7]
        test_new_x.iloc[i,22] = test_x.iloc[1 + (3*i),7]
        test_new_x.iloc[i,23] = test_x.iloc[2 + (3*i),7]
        #분무
        test_new_x.iloc[i,24] = test_x.iloc[0 + (3*i),8]
        test_new_x.iloc[i,25] = test_x.iloc[1 + (3*i),8]
        test_new_x.iloc[i,26] = test_x.iloc[2 + (3*i),8]
        #백색
        test_new_x.iloc[i,27] = test_x.iloc[0 + (3*i),9]
        test_new_x.iloc[i,28] = test_x.iloc[1 + (3*i),9]
        test_new_x.iloc[i,29] = test_x.iloc[2 + (3*i),9]
        #적색
        test_new_x.iloc[i,30] = test_x.iloc[0 + (3*i),10]
        test_new_x.iloc[i,31] = test_x.iloc[1 + (3*i),10]
        test_new_x.iloc[i,32] = test_x.iloc[2 + (3*i),10]
        #청색
        test_new_x.iloc[i,33] = test_x.iloc[0 + (3*i),11]
        test_new_x.iloc[i,34] = test_x.iloc[1 + (3*i),11]
        test_new_x.iloc[i,35] = test_x.iloc[2 + (3*i),11]
        #총광
        test_new_x.iloc[i,36] = test_x.iloc[0 + (3*i),12]
        test_new_x.iloc[i,37] = test_x.iloc[1 + (3*i),12]
        test_new_x.iloc[i,38] = test_x.iloc[2 + (3*i),12]



    test_new_x.drop(['obs_time','시간대','내부온도관측치','내부습도관측치','co2관측치','ec관측치'], axis = 1, inplace = True)

    test_x = test_new_x

    test_x = test_x.drop(test_x.filter(regex='시간당').columns, axis=1)


    test_x['하루평균온도'] = (test_x['05시내부온도관측치누적'] + test_x['19시내부온도관측치누적'] + test_x['23시내부온도관측치누적']) / 3
    test_x['하루평균습도'] = (test_x['05시내부습도관측치누적'] + test_x['19시내부습도관측치누적'] + test_x['23시내부습도관측치누적']) / 3
    test_x['하루평균co2'] = (test_x['05시co2관측치누적'] + test_x['19시co2관측치누적'] + test_x['23시co2관측치누적']) / 3
    test_x['하루평균ec'] = (test_x['05시ec관측치누적'] + test_x['19시ec관측치누적'] + test_x['23시ec관측치누적']) / 3
    test_x['하루평균분무량'] = (test_x['05시분무량누적'] + test_x['19시분무량누적'] + test_x['23시분무량누적']) / 3
    test_x['하루평균백색광'] = (test_x['05시백색광누적'] + test_x['19시백색광누적'] + test_x['23시백색광누적']) / 3
    test_x['하루평균적색광'] = (test_x['05시적색광누적'] + test_x['19시적색광누적'] + test_x['23시적색광누적']) / 3
    test_x['하루평균청색광'] = (test_x['05시청색광누적'] + test_x['19시청색광누적'] + test_x['23시청색광누적']) / 3
    test_x['하루평균총광량'] = (test_x['05시총광량누적'] + test_x['19시총광량누적'] + test_x['23시총광량누적']) / 3

    test_x = test_x.drop(test_x.filter(regex = '총광').columns, axis =1)


    test_x['hypothesis4'] = (test_x['05시ec관측치누적']+1) * (test_x['05시분무량누적']+1)
    test_x['hypothesis5'] = (test_x['19시ec관측치누적']+1) * (test_x['19시분무량누적']+1)
    test_x['hypothesis6'] = (test_x['23시ec관측치누적']+1) * (test_x['23시분무량누적']+1)
    test_x['hypothesis7'] = (test_x['하루평균ec']+1) * (test_x['하루평균분무량']+1)

    test_x['적색_+_청색05'] = (test_x['05시적색광누적']) + (test_x['05시청색광누적'])
    test_x['적색_+_청색19'] = (test_x['19시적색광누적']) + (test_x['19시청색광누적'])
    test_x['적색_+_청색23'] = (test_x['23시적색광누적']) + (test_x['23시청색광누적'])
    test_x['적색_+_청색평균'] = (test_x['하루평균적색광']) + (test_x['하루평균청색광'])

    test_x['hypothesis123'] =  test_x['hypothesis1'] + test_x['hypothesis2'] + test_x['hypothesis3']
    test_x['hypothesis12'] =  test_x['hypothesis2'] + test_x['hypothesis1']
    test_x['hypothesis23'] =  test_x['hypothesis2'] + test_x['hypothesis3']
    test_x['hypothesis13'] =  test_x['hypothesis1'] + test_x['hypothesis3']
    test_x['hypothesis123'] =  test_x['hypothesis1'] + test_x['hypothesis2'] + test_x['hypothesis3']
  

    test_x.drop(test_x.filter(regex = '백색'), axis = 1, inplace=True)

    test_x.drop(test_x.filter(regex = '청색'), axis = 1, inplace=True)

    test_x.drop(['hypothesis1', 'hypothesis2', 'hypothesis3'],axis=1,inplace=True)

    basic_col = test_x.columns
    kal_test = test_x

    for i in range(2,34):
        test_x['kf_X_'+str(i)]=0

    for j in range(0,len(test_x.Case.unique())): 
        kal = test_x
        for i in (range(len(basic_col))):
            if((i == 0) | (i == 1)):
                continue
            current=0
            sum_c=[]
            z = kal.loc[:, kal.columns[i]]
            a = []           #필터링 된 피쳐(after)
            b = []           #필터링 전 피쳐(before)
            my_filter = KalmanFilter(dim_x=2,dim_z=1) #create kalman filter
            my_filter.x = np.array([[2.],[0.]])       # initial state (location and velocity)
            my_filter.F = np.array([[1.,1.], [0.,1.]])    # state transition matrix
            my_filter.H = np.array([[1.,0.]])    # Measurement function
            my_filter.P *= 1000.                 # covariance matrix
            my_filter.R = 5                      # state uncertainty
            my_filter.Q = Q_discrete_white_noise(dim = 2,dt=.1,var=.5) # process uncertainty   
            for k in z.values:
                my_filter.predict()
                my_filter.update(k)
                x = my_filter.x
                a.extend(x[0])
                b.append(k)
            sum_c=sum_c+a
            test_x['kf_X_'+str(i)] = sum_c

    copy_test_x = test_x.copy()

    copy_test_x.drop(copy_test_x.filter(regex = 'kf').columns, axis = 1, inplace = True)

    raw_cols_te = copy_test_x.columns
    new_test_x = pd.DataFrame()
    for i in range(0,len(copy_test_x.Case.unique())):
        target = copy_test_x
        mean_arr = []
        median_arr = []

        pbar = (target.columns[:])
        for column in pbar:
            column_list = target[column].to_list()
            for i in range(7):
                if((i) == (len(column_list))):
                    break
                mean_arr.append(column_list[i])
                median_arr.append(column_list[i])

            for i in range(7, len(column_list)):
                mean_arr.append(float(np.mean(column_list[i-7:i])))
                median_arr.append(float(np.median(column_list[i-7:i])))
            target[f'{column}_mean_7'] = mean_arr
            target[f'{column}_median_7'] = median_arr
            mean_arr = []
            median_arr = []
        
        new_test_x = pd.concat([new_test_x, target], axis=0)
    test_x_moving_7 = new_test_x.drop(raw_cols_te, axis=1)

    test_x = pd.concat([test_x, test_x_moving_7], axis=1)

    raw_cols_te = copy_test_x.columns



    new_test_x = pd.DataFrame()

    for i in range(0,len(copy_test_x.Case.unique())):
        target = copy_test_x[copy_test_x['Case'] == i]
        mean_arr = []
        median_arr = []


        pbar = tqdm(target.columns[:])
        for column in pbar:
            column_list = target[column].to_list()
            for i in range(14):
                if((i) == (len(column_list))):
                    break
                mean_arr.append(column_list[i])
                median_arr.append(column_list[i])
                
            for i in range(14, len(column_list)):
                mean_arr.append(float(np.mean(column_list[i-14:i])))
                median_arr.append(float(np.median(column_list[i-14:i])))
            target[f'{column}_mean_14'] = mean_arr
            target[f'{column}_median_14'] = median_arr
            mean_arr = []
            median_arr = []
        
        new_test_x = pd.concat([new_test_x, target], axis=0)
    test_x_moving_14 = new_test_x.drop(raw_cols_te, axis=1)

    test_x = pd.concat([test_x, test_x_moving_14], axis=1)

    test_x_fil = test_x.iloc[:,1:34]


    new_test_x = pd.DataFrame()
    for i in range(0,len(copy_test_x.Case.unique())):
        target = test_x_fil
        test_merge = LPF(target,0.1,1)
        test_merge = pd.DataFrame(test_merge)
        
        new_test_x = pd.concat([new_test_x, test_merge], axis=0)
    new_test_x = new_test_x.add_suffix('_LPF')
    new_test_x = new_test_x.reset_index(drop = True)
    test_x = pd.concat([test_x,new_test_x], axis = 1)


    test_x.drop(['0_LPF'],axis = 1, inplace=True)
    test_x.drop(test_x.filter(regex = 'Case'),axis=1, inplace=True)
    return test_x

In [8]:
def make_traindata_ctgan(input_path):
    train = pd.DataFrame()
    x = pd.read_csv(input_path)
    x = x[['obs_time','내부온도관측치','내부습도관측치','co2관측치','ec관측치','시간당분무량','시간당백색광량','시간당적색광량','시간당청색광량']]
    col_list = x.columns[1:]
    for i in range(0,28) :
        day = x.iloc[24*i:24*i+24]
        time_list = day['obs_time'].unique()
        for t in range(0,len(time_list)):
            for col in col_list:
                time = time_list[t]
                value1 = day[day['obs_time']==time][col].iloc[0]
                x[col+str(time)] = value1
        nx = x.iloc[:1,9:]
        train = pd.concat([train,nx]).reset_index(drop=True)
    return train

def make_raw (df, day) :
    df_numpy = df.values
    return_arr = []
    
    for i in range(len(df_numpy)) :
            now = df_numpy[i]
            for j in range(24):
                value = np.insert(now[j*8:j*8+8],0, int(day)) # dat
                value = np.insert(value , 1 ,int(j)) # obs_time
                value = np.insert(value , 0 ,int(i)) # case
                return_arr.append(value.tolist())

    return_df = pd.DataFrame(np.array(return_arr),columns = ['Case','DAT', 'obs_time', '내부온도관측치', '내부습도관측치','co2관측치', 
                                        'ec관측치', '시간당분무량', '시간당백색광량',  '시간당적색광량',  '시간당청색광량' ])
    
    return_df['시간당총광량'] = return_df['시간당백색광량'] + return_df['시간당적색광량'] + return_df['시간당청색광량']
    
    # 중앙값 차이 값으로 값 변환
    return_df['내부온도관측치'] =  return_df['내부온도관측치']  - 2
    return_df['시간당분무량'] = return_df['시간당분무량'] + 1251.865000
    return_df['ec관측치'] = return_df['ec관측치'] +1.7

    cumsum_list = ['일간누적분무량', '일간누적백색광량', '일간누적적색광량', '일간누적청색광량', '일간누적총광량']
    per_time_list = ['시간당분무량', '시간당백색광량', '시간당적색광량', '시간당청색광량', '시간당총광량']

    for i in range(0, 5):
        col1 = cumsum_list[i]
        col2 = per_time_list[i]
        return_df[col1] = 0
        return_df[col1] = return_df.groupby((return_df.obs_time == 0).cumsum()).agg(col2).cumsum()

    return_df = return_df[['Case', 'DAT', 'obs_time', '내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량',
             '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량',
             '일간누적청색광량', '시간당총광량', '일간누적총광량']]

    return return_df

# CTGAN

In [9]:
import numpy as np
import random
import os
import torch

def seed_everything(seed: int=0):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)               # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True      # type: ignore

''' sample '''
# seed_everything(0)
from ctgan import CTGAN

In [11]:
# [path] 생성한 데이터샘플 저장
generated_path = "/content/drive/MyDrive/Project_L/ctgan_day0_27/"

# [path] ctgan에 넣을 데이터 
input_path1 = '/content/drive/MyDrive/dacon/gan/상추/train_input/CASE_21.csv' 

In [14]:
def augmentation(input_path1, save_path):
        
    # [flatten] real data -> ctgan input data
    raw_data1 = make_traindata_ctgan(input_path1)

    input_data = raw_data1.iloc[:7]

    model = CTGAN()
    model.fit(input_data, epochs=50)
    output_data = model.sample(50)
    model.save("/content/drive/MyDrive/Project_L/gen_model7")

    for i in range(0,7):
        dayi = make_raw(output_data,i)
        dayi.to_csv(os.path.join(generated_path, f'generate_day_{i}_50_R.csv'), index=False)

    input_data = raw_data1.iloc[7:14]
    model = CTGAN()
    model.fit(input_data, epochs=50)
    output_data = model.sample(50)
    model.save("/content/drive/MyDrive/Project_L/gen_model14")

    for i in range(7,14):
        dayi = make_raw(output_data,i)
        dayi.to_csv(os.path.join(generated_path, f'generate_day_{i}_50_R.csv'), index=False)

    input_data = raw_data1.iloc[14:21]
    model = CTGAN()
    model.fit(input_data, epochs=50)
    output_data = model.sample(50)
    model.save("/content/drive/MyDrive/Project_L/gen_model21")

    for i in range(14,21):
        output_data = model.sample(50)
        dayi = make_raw(output_data,i)
        dayi.to_csv(os.path.join(generated_path, f'generate_day_{i}_50_R.csv'), index=False)

    input_data = raw_data1.iloc[21:]
    model = CTGAN()
    model.fit(input_data, epochs=50)
    output_data = model.sample(50)
    model.save("/content/drive/MyDrive/Project_L/gen_model28")

    for i in range(21,28):
        output_data = model.sample(50)
        dayi = make_raw(output_data,i)
        dayi.to_csv(os.path.join(generated_path, f'generate_day_{i}_50_R.csv'), index=False)    

In [15]:
augmentation(input_path1,generated_path)

In [ ]:
# 예측모델 로드
predict_model = TabularPredictor.load("/content/drive/MyDrive/Project_L/최종예측모델", require_version_match=False)

# 0일차 예측
gen_data = pd.read_csv('/content/drive/MyDrive/Project_L/ctgan_day0_27/generate_day_0_50_R.csv')  
max_list = []
for i in range(50):
    gen_data_new = gen_data[i*24:(i+1)*24]
    gen_data_new['Case']=0
    gan_data = makedatasets(gen_data_new)
    y_pred = predict_model.predict(gan_data)
    max_list.append(y_pred.max())
max_list = pd.DataFrame(max_list)
idx_max = max_list.idxmax()
gen_dat = gen_data[int(idx_max.values)*24:(int(idx_max.values)+1)*24]
gen_dat['DAT'] = 0
print(max_list.max())
# fix data 경로
gen_dat.to_csv("/content/drive/MyDrive/Project_L/fix.csv", index=False)

In [ ]:
predict_model = TabularPredictor.load("/content/drive/MyDrive/Project_L/최종예측모델", require_version_match=False)

for day in range(1, 28):
    max_0 = pd.read_csv("/content/drive/MyDrive/Project_L/fix.csv")
    print(f'{day}일차 Start')
    gen_data = pd.read_csv(f'/content/drive/MyDrive/dacon/상/generate_day_{day}_50_R.csv')  # 동적변경
    max_list = []
    for i in range(50):
        gen_data_new = gen_data[i*24:(i+1)*24]
        gen_data_new = gen_data_new.reset_index(drop=True)
        max_0['Case'] = i
        gen_data_new['Case'] = i
        gen_data_new['DAT'] = day 
        gan_data = make_data(gen_data_new, max_0, day)
        y_pred = predict_model.predict(gan_data)
        max_list.append(y_pred.max())
    max_list = pd.DataFrame(max_list)
    print(max_list.max())
    idx_max = max_list.idxmax()
    gen_dat = gen_data[int(idx_max.values)*24:(int(idx_max.values)+1)*24]
    gen_dat['DAT'] = day
    con_dat = pd.concat([max_0, gen_dat], axis=0)
    con_dat.to_csv("/content/drive/MyDrive/Project_L/fix.csv", index=False)
    print(f'{day}일차 End------------------------------')